First I'll try to not use Selenium and see if scraping is possible simply through using a request.

In [1]:
import requests

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
}

In [3]:
res = requests.get("https://twitter.com/GTNUK1", headers=headers, timeout=5)

In [4]:
res.status_code

200

After some scavenging I've found out the CSS selectors used by Twitter for the relevant data (i.e. Bio, Following Count, Followers Count, Location, Website).

In [24]:
CONTAINER = "[data-testid='primaryColumn']"
selectors = {
    "Bio": "[data-testid='UserDescription']",
    "Following": ".r-bcqeeo.r-qvutc0.r-37j5jr.r-a023e6.r-rjixqe.r-16dba41.r-1loqt21 .r-bcqeeo.r-qvutc0.r-poiln3",
    "Followers": ".r-bcqeeo.r-qvutc0.r-37j5jr.r-a023e6.r-rjixqe.r-16dba41.r-1loqt21 .r-1b43r93.r-1cwl3u0.r-b88u0q",
    "Location": "[data-testid='UserLocation']",
    "Website": "[data-testid='UserUrl']",
}

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(res.text, "html.parser")

In [10]:
information = []


soup.select(selectors[key]))

[]
[]
[]
[]
[]


Clearly this doesn't work...  
Workaround: Use selenium instead.

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [21]:
options = webdriver.FirefoxOptions()
options.add_argument("--headless")
driver = webdriver.Firefox()

In [30]:
driver.get("https://twitter.com/GTNUK1")
WebDriverWait(driver, 1).until(
  EC.presence_of_element_located((By.CSS_SELECTOR, selectors["Followers"]))
)
container = driver.find_element(By.CSS_SELECTOR, CONTAINER).get_attribute("innerHTML")
driver.quit()

In [25]:
soup = BeautifulSoup(container, "html.parser")

In [28]:
info = {}

for key in selectors:
  info[key] = soup.select_one(selectors[key]).text

In [29]:
info

{'Bio': 'Providing Entertainment & Travel to Commercial Radio. Reaching 28.9M weekly listeners. Winners of The Arqiva National Sales Team of the Year 2010, 2011 & 2016',
 'Following': '460',
 'Followers': '460',
 'Location': 'London, England',
 'Website': 'gtn.uk.com/index.php'}